A Decision Tree is a supervised machine learning algorithm that is used for both classification and regression tasks. It works by splitting the dataset based on feature values. This is a common process in a humans decision making e.g. I want to buy a new pair of shoes, first I separate by brand, then from that brand the styles I like, and finally from those styles which ones are in my price range.

The structure follows:

* Internetal node which represents the decision rule on a feature (for each data point feature =< threshold).
* Branch representing the outcome of that rule (left = true, right = false).
* Leaf node or terminal node which represents the final output. For classification this is the most common class of samples in that node. For regressions this is an average of the target values in that node.


##Explanation

With this structure in mind, see below the defined `Node` helper class which is used to represent a single node in the tree and it's information.    
    
    
    class Node:
        '''
        Arguments:
        feature -- which column the threshold will refer to
        threshold -- the splitting parameter
        left -- left child node
        right -- right child node
        value -- prediction, exist only if it is leaf node.
        '''
        def __init__(self, feature=None, threshold=None, left=None, right=None, value=None):
            self.feature = feature
            self.threshold = threshold
            self.left = left
            self.right = right
            self.value = value

Before creating the Decision Tree class, I will go through each of its functions.

1) Initialise the tree's parameters.
* `min_samples_split` - the minimum number of samaples required to split and internal node. Prevents splitting without too few nodes which avoids overfitting.
* `max_depth` - Maxmimum depth or max number of recursive splits. Prevents overfitting and controls computation cost.
* `measure_name` - Defines what criterion to ues to evaluate the quality of a decision rule.
* `root` - First node in the tree


      def __init__(self, min_samples_split=2, max_depth=100, measure_name="entropy", n_feats=None):
          self.min_samples_split = min_samples_split
          self.max_depth = max_depth
          self.measure_name = measure_name
          self.root = None

2) Start decision making

`fit` - call to start growing the tree

    def fit(self, X, y):
      self.root = self._grow_tree(X, y, depth=0)

`_grow_tree` - recursively find best split to grow tree until leaf node.

* Check depth and number of samples fulfill requirements, if not add the most common value to the Node class's value (becomes leaf node).
* If requirements are fulfilled find the best split of the input data.
* Based on the best split, create then left and right splits, and grow the tree again.
* Add the details of this split to the node class.


      def _grow_tree(self, X, y, depth=0):
        n_samples, n_features = X.shape
        n_labels = len(np.unique(y))

        # stopping criteria
        if depth >= self.max_depth or n_labels == 1 or n_samples < self.min_samples_split:
            leaf_value = Counter(y).most_common(1)[0][0]
            return Node(value=leaf_value)

        # greedily select the best split according to information gain
        best_feat, best_thresh = self._best_split(X, y, feat_idxs)

        # grow the children that result from the split
        left_idxs, right_idxs = self._split(X[:, best_feat], best_thresh)
        left = self._grow_tree(X[left_idxs, :], y[left_idxs], depth+1)
        right = self._grow_tree(X[right_idxs, :], y[right_idxs], depth+1)
        return Node(best_feat, best_thresh, left, right)

Splits are not random, they are based on which split will reduce the uncertainty of the target variable the most.

`_best_split` - loop through every feature and data point within that feature to see which combination results in the best information gain; data point being at which threshold in that feature to split the inputs.

    def _best_split(self, X, y, feat_idxs):
        best_gain = -1
        split_idx, split_threshold = None, None
        for feat_idx in feat_idxs:
            X_column = X[:, feat_idx]
            candidate_thresholds = np.unique(X_column)
            for threshold in candidate_thresholds:
                gain = self._information_gain(y, X_column, threshold)

                if gain > best_gain:
                    best_gain = gain
                    split_idx = feat_idx
                    split_threshold = threshold

        return split_idx, split_threshold

3) Measuring the best split

There are different methods for quantifying information gain, and these differ between classification and regression trees. I will focus on Gini and Entropy for classification, but a regression tree may use Variance or Mean Squared Error reduction.

For entropy, the proportion of samples in that class $P(c)$ is multiplied by the log base 2 of the class $log_2 p(c)$. This is done separtely for classes (above and below the threshold) and summed.

$Entropy(feature) = - \displaystyle\sum_{c \in \text{classes}} p(c) \log_2 p(c)$

Gini impurity measures how often a randomly chosen data point would be misclassified if randomly labeled following the class distribution. In the equation we sum the squared probability $p^2$ of a data point belonging to a class and minus that sum from 1.

$Gini\ impurity = 1  -\displaystyle\sum_{c \in \text{classes}} p(c) \log_2 p(c)$

The methods described above measure the impurity of the split. To get the information gain we measure the difference in this impurity before (in the parent node) and after (in the child nodes) the split.

`_information_gain` does this:
* compute the impurity of parent node `parent_entropy`.
* splt the node based on the feature and threshold provided by the `_best_split` for loop.
* compute the impurity of the child nodes before averaging by weight and then summing together.
*calculate the information gain = parent entropy - children entropy.

      def _information_gain(self, y, X_column, split_threshold):

        parent_entropy = information_measure(y, self.measure_name)

        # generate split
        left_idxs, right_idxs = self._split(X_column, split_threshold)

        # if all elements fall in one side or the other the split has no effect
        if len(left_idxs) == 0 or len(right_idxs) == 0:
          return 0

        # compute the weighted average of the entropy for the children
        n = len(y)
        left_child_entropy = information_measure(y[left_idxs],self.measure_name)
        right_child_entropy = information_measure(y[right_idxs],self.measure_name)
        left_weight = len(left_idxs) / n
        right_weight = len(right_idxs) / n
        children_entropy =  left_weight * left_child_entropy + right_weight * right_child_entropy

        # information gain as the difference in entropy before and after split
        information_gain = parent_entropy - children_entropy
          return information_gain

      def _split(self, X_column, split_threshold):
          # returns 2 arrays, one with values <= threshold and the other > thershold
          left_idxs = np.argwhere(X_column <= split_threshold).flatten()
          right_idxs = np.argwhere(X_column > split_threshold).flatten()
          return left_idxs, right_idxs

4) Predict

After fitting the best splits to our data through training, we now want to predict unseen data. `_traverse_tree` does this by starting with the tree root, and calling the corresponding nodes of the parents left/right split. If the node has a value it is a leaf node and returns the result, otherwise it continues.

    def _traverse_tree(self, x, node):
        '''
        Recursively traverse the tree.

        Arguments:
        x is the input vector that needs to be classified.
        node is the current node.
        '''
        # if it is a leaf node returns its value
        if node.value is not None:
            return node.value

        if x[node.feature] <= node.threshold:
            return self._traverse_tree(x, node.left)
        return self._traverse_tree(x, node.right)



##Implementation

In [1]:
# Implementation
import numpy as np
from collections import Counter

#Impurity calculations
def entropy(y):
    counts = np.bincount(y)
    percentages  = counts / len(y)
    entropy = 0

    for p in percentages:
      if p > 0:
        entropy += p*np.log2(p)
    return -entropy

def gini_impurity(y):
    counts = np.bincount(y)
    probabilities = counts / len(y)
    impurity = 1 - np.sum(probabilities**2)
    return impurity

def information_measure(y, name):
    if name == "entropy":
        return entropy(y)
    elif name == "gini":
        return gini_impurity(y)
    else:
        print("invalid measure name")

#Store node information
class Node:
    '''
    Helper class holding node information.

    Arguments:
    feature -- which column the threshold will refer to
    threshold -- the splitting parameter
    left -- left child node
    right -- right child node
    value -- label of the node, exist only if it is leaf node.
    '''
    def __init__(self, feature=None, threshold=None, left=None, right=None, value=None):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value

class DecisionTree:

    def __init__(self, min_samples_split=2, max_depth=100, measure_name="entropy", n_feats=None):
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.n_feats = n_feats
        self.root = None
        self.measure_name = measure_name

    def fit(self, X, y):
        # if not specified the tree will use all the features
        if self.n_feats != None:
          self.n_feats = min(self.n_feats, X.shape[1])
        else:
          self.n_feats = X.shape[1]

        self.root = self._grow_tree(X, y)

    def predict(self, X):
        return np.array([self._traverse_tree(x, self.root) for x in X])

    def _grow_tree(self, X, y, depth=0):
        n_samples, n_features = X.shape
        n_labels = len(np.unique(y))

        # stopping criteria
        if depth >= self.max_depth or n_labels == 1 or n_samples < self.min_samples_split:
            leaf_value = Counter(y).most_common(1)[0][0]
            return Node(value=leaf_value)

        # pick a random subset of the features if n_feats is smaller than n_features (used by Random Forests)
        feat_idxs = np.random.choice(n_features, self.n_feats, replace=False) # random forest

        # greedily select the best split according to information gain
        best_feat, best_thresh = self._best_split(X, y, feat_idxs)

        # grow the children that result from the split
        left_idxs, right_idxs = self._split(X[:, best_feat], best_thresh)
        left = self._grow_tree(X[left_idxs, :], y[left_idxs], depth+1)
        right = self._grow_tree(X[right_idxs, :], y[right_idxs], depth+1)
        return Node(best_feat, best_thresh, left, right)

    def _best_split(self, X, y, feat_idxs):
        best_gain = -1
        split_idx, split_threshold = None, None
        #Find best feature and threshold combination
        for feat_idx in feat_idxs:
            X_column = X[:, feat_idx]
            candidate_thresholds = np.unique(X_column)
            for threshold in candidate_thresholds:
                gain = self._information_gain(y, X_column, threshold)

                if gain > best_gain:
                    best_gain = gain
                    split_idx = feat_idx
                    split_threshold = threshold

        return split_idx, split_threshold

    def _information_gain(self, y, X_column, split_threshold):

        parent_entropy = information_measure(y, self.measure_name)

        # generate split
        left_idxs, right_idxs = self._split(X_column, split_threshold)

        # if all elements fall in one side or the other the split has no effect
        if len(left_idxs) == 0 or len(right_idxs) == 0:
          return 0

        # compute the weighted average of the entropy for the children
        n = len(y)
        left_child_entropy = information_measure(y[left_idxs],self.measure_name)
        right_child_entropy = information_measure(y[right_idxs],self.measure_name)
        left_weight = len(left_idxs) / n
        right_weight = len(right_idxs) / n
        children_entropy =  left_weight * left_child_entropy + right_weight * right_child_entropy

        # information gain as the difference in entropy before and after split
        information_gain = parent_entropy - children_entropy
        return information_gain

    def _split(self, X_column, split_threshold):
        # returns 2 arrays, one with all the indices of the values smaller or equal than the threshold and one with all the indices that are greater
        left_idxs = np.argwhere(X_column <= split_threshold).flatten()
        right_idxs = np.argwhere(X_column > split_threshold).flatten()
        return left_idxs, right_idxs

    def _traverse_tree(self, x, node):
        '''
        Recursively traverse the tree.

        Arguments:
        x is the input vector that needs to be classified.
        node is the current node.
        '''
        # if it is a leaf node returns its value
        if node.value is not None:
            return node.value

        if x[node.feature] <= node.threshold:
            return self._traverse_tree(x, node.left)
        return self._traverse_tree(x, node.right)

In [7]:
# Import data
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import time

data = load_breast_cancer()
X = data.data
y = data.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

start = time.time_ns()
# Default uses entropy for impurity measure
tree = DecisionTree(max_depth=12)
tree.fit(X_train, y_train)
end = time.time_ns() - start
# Computation duration (seconds)
print(f"Processing time (s): {end/1000000000}")

# Precision = True Positives /(True Positives + False Positives). i.e if I say class 0 what is the probability I am correct
# Recall = TP / (TP + FN). i.e what proportion of class 0 did I predict correctly
# f1-score = 2 * ((precision * recall)/(precision + recall))
# Support = Number of true samples in that class

y_pred = tree.predict(X_test)
print(classification_report(y_test, y_pred))

Processing time (s): 1.416699012
              precision    recall  f1-score   support

           0       0.95      0.90      0.92        80
           1       0.95      0.97      0.96       148

    accuracy                           0.95       228
   macro avg       0.95      0.94      0.94       228
weighted avg       0.95      0.95      0.95       228



Compared with sklearn imported model.

In [3]:
from sklearn.tree import DecisionTreeClassifier

sklearn_tree = DecisionTreeClassifier(max_depth=12)
sklearn_tree.fit(X_train, y_train)
y_pred_sklearn = sklearn_tree.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.95      0.90      0.92        80
           1       0.95      0.97      0.96       148

    accuracy                           0.95       228
   macro avg       0.95      0.94      0.94       228
weighted avg       0.95      0.95      0.95       228



##Random Forest


The expantion to a Randon Forest is simple. It is an ensemble of decision trees with predictions being made on a majority vode of these trees/

    for tree in forest:
        randomly sample a portion of data for tree subset
        call decision tree on that subset
        append tree to self.trees
    Count votes and return max

In my code a prediction is made by majority vote of the individual tree outputs. However, other methods to do this are weighted averages based on training performance, or averaging probabilities instead of hard label predictions.

In [5]:
class RandomForest:
    def __init__(self, n_trees, portion=1, min_samples_split=2, max_depth=100,n_feats=None):
        self.n_trees = n_trees
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.n_feats = n_feats        # Number of features to condiser at each split
        self.portion = portion        # Portion of data to put into a subset
        self.trees = []

    def fit(self, X, y):
        self.trees = []
        for _ in range(self.n_trees):
            tree = DecisionTree(
                min_samples_split=self.min_samples_split,
                max_depth=self.max_depth,
                n_feats=self.n_feats,
            )
            #bootstrapping
            n_samples = X.shape[0]
            subset_size = int(self.portion * n_samples)
            #there are different ways to sample rather than random
            indicies = np.random.choice(n_samples, subset_size, replace=True)
            Xs = X[indicies]
            ys = y[indicies]

            #train
            tree.fit(Xs, ys)
            self.trees.append(tree)

    def predict(self, X):
        votes = []
        y_pred = []

        for tree in self.trees:
            votes.append(tree.predict(X))

        votes = np.swapaxes(np.array(votes), 0, 1)
        for v in votes:
            values, counts = np.unique(v, return_counts=True)
            index = np.argmax(counts)
            y_pred.append(values[index])
        return np.array(y_pred)

In [9]:
RF = RandomForest(n_trees = 5, portion = 0.5, max_depth = 12, n_feats = 20)

RF.fit(X_train, y_train)
y_pred = RF.predict(X_test)
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.95      0.94      0.94        80
           1       0.97      0.97      0.97       148

    accuracy                           0.96       228
   macro avg       0.96      0.96      0.96       228
weighted avg       0.96      0.96      0.96       228



In [25]:
# test against sklearn import
# random sampling causes results to differ when re-running
from sklearn.ensemble import RandomForestClassifier

sklearn_RF = RandomForestClassifier(n_estimators=5, max_depth=12)
sklearn_RF.fit(X_train, y_train)
y_pred_sklearn = sklearn_RF.predict(X_test)
print(classification_report(y_test, y_pred_sklearn))

              precision    recall  f1-score   support

           0       0.96      0.95      0.96        80
           1       0.97      0.98      0.98       148

    accuracy                           0.97       228
   macro avg       0.97      0.96      0.97       228
weighted avg       0.97      0.97      0.97       228

